In [1]:
import os
import subprocess

# Directory where the PDB files are located
pdb_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"

# Base directory where you want to create folders
base_scr_dir = "/home/hp/results/humansec"

# Directories for the input files
run_dir = "/home/hp/nayanika/github/GPX6/cluster_scripts"
genrelax_dir = "/home/hp/nayanika/github/GPX6/input"
topology_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"
fep_dir = "/home/hp/nayanika/github/GPX6/input/fep"  # Correct FEP directory path

# Create the mousecys directory if it does not exist
os.makedirs(base_scr_dir, exist_ok=True)

# Loop through each PDB file in the directory
for pdb_file in os.listdir(pdb_dir):
    if pdb_file.endswith(".pdb"):
        # Get the full path of the PDB file
        pdb_file_path = os.path.join(pdb_dir, pdb_file)
        
        # Extract the system name from the PDB file (removing the extension)
        system_name = os.path.basename(pdb_file).replace(".pdb", "")

        # Define the topology file path
        topology_file = os.path.join(topology_dir, f"{system_name}.top")
        
        # Check if the topology file exists
        if not os.path.isfile(topology_file):
            print(f"No topology file found for {system_name} at {topology_file}")
            continue

        # Create a directory for the system in the base output directory
        system_dir = os.path.join(base_scr_dir, system_name)
        os.makedirs(system_dir, exist_ok=True)

        # Check if the .pdb file exists
        if os.path.isfile(pdb_file_path):
            # Construct the command to run q_genrelax.py
            command = [
                "q_genrelax.py", 
                os.path.join(genrelax_dir, "selgenrelax.proc"),
                "--top", topology_file,
                "--pdb", pdb_file_path,
                "--fep", os.path.join(fep_dir, "GPX6_wthumansec.fep"),
                "--outdir", os.path.join(system_dir, "minim"),
                "--rs", os.path.join(run_dir, "run_qdyn_5.sh")
            ]
            
            # Run the command
            try:
                subprocess.run(command, check=True)
                print(f"Successfully processed {pdb_file_path} with topology {topology_file}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing {pdb_file_path}: {e}")
        else:
            print(f"No .pdb file found at {pdb_file_path}")


# These placeholders will be replaced with atom indices: $1.N$, $195.O$, $49.SE$, $198.O1$, $83.OE1$, $198.O1$, $49.SE$, $49.HG1$, $198.O1$, $198.H1$, $83.OE1$, $198.H1$, $198.O1$, $49.HG1$
# Restraining to: 'topology'
# Replacing FEP file placeholders...
# Generated random seed in step 1: 152191
# Generated random seed in step 2: 577944
# Generated random seed in step 3: 247624
# Generated random seed in step 4: 419190
# Generated random seed in step 5: 517264
# Generated random seed in step 6: 804909
# Generated random seed in step 7: 280340
# Generated random seed in step 8: 794745
# Generated random seed in step 9: 961395
# Generated random seed in step 10: 232311
# Generated random seed in step 11: 792084
# Generated random seed in step 12: 345163
# Created inputs relax_001.inp - relax_012.inp
# Overridden parameters:
# 6: md/stepsize:0.1->0.5, md/bath_coupling:0.1->0.5, intervals/output:10->1000, intervals/trajectory:10->1000, intervals/temperature:10->1000
# 12: md/stepsize:0.1-

Traceback (most recent call last):
  File "/home/hp/nayanika/github/qtools/qscripts-cli/q_genrelax.py", line 119, in <module>
    gen_inps = genrelax(**kwargs)
  File "/home/hp/nayanika/github/qtools/packages/Qpyl/qgeninp.py", line 136, in genrelax
    qstruct = QStruct(pdb_file, "pdb", ignore_errors=ignore_errors)
  File "/home/hp/nayanika/github/qtools/packages/Qpyl/core/qstructure.py", line 103, in __init__
    FILE_TYPES[self.filetype](filename)
  File "/home/hp/nayanika/github/qtools/packages/Qpyl/core/qstructure.py", line 206, in _read_pdb
    x, y, z = map(float, (line[30:38], line[38:46], line[46:54]))
ValueError: could not convert string to float: '1  29.11'


# These placeholders will be replaced with atom indices: $1.N$, $195.O$, $49.SE$, $198.O1$, $83.OE1$, $198.O1$, $49.SE$, $49.HG1$, $198.O1$, $198.H1$, $83.OE1$, $198.H1$, $198.O1$, $49.HG1$
# Restraining to: 'topology'
# Replacing FEP file placeholders...
# Generated random seed in step 1: 29497
# Generated random seed in step 2: 493695
# Generated random seed in step 3: 187952
# Generated random seed in step 4: 550159
# Generated random seed in step 5: 161248
# Generated random seed in step 6: 390615
# Generated random seed in step 7: 846745
# Generated random seed in step 8: 621935
# Generated random seed in step 9: 447391
# Generated random seed in step 10: 820903
# Generated random seed in step 11: 811848
# Generated random seed in step 12: 271969
# Created inputs relax_001.inp - relax_012.inp
# Overridden parameters:
# 6: md/stepsize:0.1->0.5, md/bath_coupling:0.1->0.5, intervals/output:10->1000, intervals/trajectory:10->1000, intervals/temperature:10->1000
# 12: md/stepsize:0.1->